### **Training LegalBERT**

In [ ]:
pip install transformers

In [ ]:
pip install wandb

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00


In [ ]:
import requests
import json
import pandas as pd
from datasets import Dataset

# Step 1: Download the JSON file
url = "https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/train.json"
response = requests.get(url)
json_data = response.json()


In [ ]:
def json_to_dataframe(json_data):
    data = []
    for document in json_data:
        doc_id = document.get("id")
        for annotation in document.get("annotations", []):
            for result in annotation.get("result", []):
                segment = {
                    'doc_id': doc_id,
                    'text': result['value'].get('text'),
                    'label': result['value'].get('labels', [None])[0]  # Get the first label if available
                }
                data.append(segment)
    return pd.DataFrame(data)

In [ ]:
# Convert JSON to DataFrame
df = json_to_dataframe(json_data)
print(df.head(100))

    doc_id                                               text     label
0     1735        IN THE HIGH COURT OF KARNATAKA,\n       ...  PREAMBLE
1     1735  \n\n      BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...  PREAMBLE
2     1735  This Criminal Appeal is filed under Section 37...  PREAMBLE
3     1735  \n\n       This appeal coming on for hearing t...  PREAMBLE
4     1735  \n       Heard the learned Counsel for the app...      NONE
..     ...                                                ...       ...
95    4183                                          \nJUSTICE  PREAMBLE
96    4183  .\n\n         THIS I.T.A. COMING ON FOR HEARIN...  PREAMBLE
97    4183          \nS. SUJATHA J. DELIVERED THE FOLLOWING:-  PREAMBLE
98    4183       \n\n                                JUDGMENT  PREAMBLE
99    4183  \nMr. E.I.Sanmathi Adv. for Appellants- Revenu...      NONE

[100 rows x 3 columns]


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)
# Convert DataFrame to Hugging Face Dataset again
dataset = Dataset.from_pandas(df)

{'ANALYSIS': 0, 'ARG_PETITIONER': 1, 'ARG_RESPONDENT': 2, 'FAC': 3, 'ISSUE': 4, 'NONE': 5, 'PREAMBLE': 6, 'PRE_NOT_RELIED': 7, 'PRE_RELIED': 8, 'RATIO': 9, 'RLC': 10, 'RPC': 11, 'STA': 12}


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import load_dataset, Dataset
import torch
import pandas as pd
from transformers import AutoModel


In [ ]:
# Load the tokenizer and model
model_name = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(set(df['label'])))
# Step 2: Convert DataFrame to Hugging Face Dataset and encode labels
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(df)

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Map:   0%|          | 0/28986 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict
# Split the dataset into train, test, and eval sets
# Adjust the splits according to your needs
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_test_dataset = split_dataset['test'].train_test_split(test_size=0.5)

datasets = DatasetDict({
    'train': split_dataset['train'],
    'test': train_test_dataset['test'],
    'eval': train_test_dataset['train']
})

In [ ]:
#To see the train set
import pandas as pd

# Convert the Hugging Face Dataset to a Pandas DataFrame
train_df = datasets['train'].to_pandas()

# Display the DataFrame
print(train_df)

       doc_id                                               text  label  \
0        1713  So, every person, who is mentally diseased, is...      0   
1        4174  \n 2. As the facts unfurl the details are that...      3   
2        4097  In the question referred to the High Court, th...      0   
3        4112  4th defendant, taking undue advantage of his p...      3   
4        1773  \n 5.  In support of its case prosecution has ...      0   
...       ...                                                ...    ...   
23183    4194  It has to be intrinsically connected to 'educa...      8   
23184    1751  The courts are further expected to do justice ...      8   
23185    4109  By the exercise of the power to assess fresh s...      0   
23186    4115  However, under Rule 5 of the CENVAT Credit Rul...      1   
23187    4070               R. Ramachandran for the Respondents.      5   

                                               input_ids  \
0      [101, 340, 115, 1425, 322, 115, 

In [ ]:
train_test_dataset

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2899
    })
    test: Dataset({
        features: ['doc_id', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2899
    })
})

In [ ]:
# Step 3: Setup Training Arguments
training_args = TrainingArguments(
    output_dir="./testresultsLegalBERT",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    save_total_limit=1,
    save_steps=100,
    logging_dir="./logs",
    report_to="wandb"  # Ensure W&B is properly set up, or remove this line
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets['train'],
    eval_dataset=datasets['eval']
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,1.621100
1000,1.142400
1500,0.996600
2000,0.813100


TrainOutput(global_step=2172, training_loss=1.1171182804564306, metrics={'train_runtime': 6755.9764, 'train_samples_per_second': 10.297, 'train_steps_per_second': 0.321, 'total_flos': 1.828697182969037e+16, 'train_loss': 1.1171182804564306, 'epoch': 2.996895481200414})

In [ ]:
# Save model and tokenizer
model.save_pretrained("/content/drive/MyDrive/to be delete/TrainedlegalBERT")
tokenizer.save_pretrained("/content/drive/MyDrive/to be delete/TrainedlegalBERT")

# Precision, Recall, F1 score, Accuracy (Evaluating Model)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer

# Define a compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# If the model is already trained, just initialize the Trainer with the same model
trainer = Trainer(
    model=model,  # Use your already trained model
    #args=training_args, #removed because i was testing the original model
    eval_dataset=datasets['eval'],  # Use your evaluation dataset
    compute_metrics=compute_metrics  # Include the metrics function
)

# Evaluate the model
results = trainer.evaluate()

# Print the evaluation results
print(results)



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: engineersaloni159 (salonijnu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 2.526451826095581, 'eval_model_preparation_time': 0.0051, 'eval_accuracy': 0.05036219385995171, 'eval_precision': 0.13964890058874457, 'eval_recall': 0.05036219385995171, 'eval_f1': 0.032593494945360305, 'eval_runtime': 78.5334, 'eval_samples_per_second': 36.914, 'eval_steps_per_second': 4.622}


In [ ]:
model


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer

# Define a compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Initialize the Trainer with the already trained model
trainer = Trainer(
    model=model,  # Use your already trained model
    args=training_args,
    eval_dataset=datasets['test'],  # Use your test dataset
    compute_metrics=compute_metrics  # Include the metrics function
)

# Evaluate the model on the test dataset
results = trainer.evaluate()

# Print the evaluation results
print("Test Set Evaluation Results:")
print(results)


confusion mtrix


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from transformers import Trainer

# Define a function to compute predictions on the test set
def get_predictions(trainer, test_dataset):
    predictions = trainer.predict(test_dataset)
    preds = predictions.predictions.argmax(axis=-1)  # Get the predicted labels
    labels = predictions.label_ids  # Get the true labels
    return preds, labels

# Get predictions from the model
preds, labels = get_predictions(trainer, datasets['test'])

# Compute the confusion matrix
conf_matrix = confusion_matrix(labels, preds)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Make predictions
predictions = trainer.predict(tokenized_dataset)
predicted_labels = predictions.predictions.argmax(-1)

# Print predicted labels
print(predicted_labels)

[3 6 6 ... 0 6 0]


# save this model on huggingface

In [ ]:
pip install huggingface_hub


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
from huggingface_hub import HfApi

# Define repository name and create repo on the hub
repo_name = "In_legal_distillbert"  # Choose a unique name
HfApi().create_repo(repo_name)

# Upload the model
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/engineersaloni159/In_legal_distillbert/commit/ebc4c10b7b1040053b267cba98b9a42ee84dcb11', commit_message='Upload tokenizer', commit_description='', oid='ebc4c10b7b1040053b267cba98b9a42ee84dcb11', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.save_pretrained(repo_name, push_to_hub=True)
tokenizer.save_pretrained(repo_name, push_to_hub=True)


('In_legal_distillbert/tokenizer_config.json',
 'In_legal_distillbert/special_tokens_map.json',
 'In_legal_distillbert/vocab.txt',
 'In_legal_distillbert/added_tokens.json',
 'In_legal_distillbert/tokenizer.json')

# Manually Evaluation of model

In [ ]:
pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("engineersaloni159/legalBERT_for_rhetorical_role_labeling")
model = AutoModelForSequenceClassification.from_pretrained("engineersaloni159/legalBERT_for_rhetorical_role_labeling")


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/702k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Ensure model is in evaluation mode
model.eval()

In [ ]:
#run this cell if model is directly called from  huggingface
from sklearn.preprocessing import LabelEncoder

# Assuming you already have this
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Print the mapping of labels to numbers
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

NameError: name 'df' is not defined

In [ ]:
# Assuming label_encoder is the one you used for encoding labels earlier

# Example sentence
sentence = "Appeal by special leave from the judgment and order dated September 11, 1964 of the Madras High Court in Tax Case No. 181 of 1962."
# Tokenize the sentence
inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)

# The predictions are in the form of logits, you can convert them to probabilities
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1)

# Convert the predicted class number to the original label
predicted_label = label_encoder.inverse_transform(predicted_class.cpu().numpy())[0]

# Print the predicted label
print(f"Predicted label: {predicted_label}")


Predicted label: NONE


In [ ]:
print("Logits:", logits)


Logits: tensor([[ 2.2846,  0.2215, -0.3713, -0.5981, -0.7258, -0.8616,  0.2079, -1.1513,
          0.6050, -0.1885, -0.6332, -0.5202, -0.2322]])
